# Import Libraries


In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)
pd.options.mode.copy_on_write = True

In [2]:
class NestedData:
    def __init__(self, data_dict):
        for key, value in data_dict.items():
            setattr(self, key, value)

    def __repr__(self):
        attrs = ", ".join([f"{k}={v}" for k, v in self.__dict__.items()])
        return f"NestedData({attrs})"


def nest(df, nested_column_name, columns_to_nest):
    """
    Create a nested column with dot notation access.
    """

    def create_test_object(row):
        data_dict = {col: row[col] for col in columns_to_nest if col in row.index}
        return NestedData(data_dict)

    # Create the nested column
    df[nested_column_name] = df.apply(create_test_object, axis=1)

    # Remove the original columns
    df = df.drop(columns=columns_to_nest)

    return df

# School Year Calendar


In [3]:
school_year = pd.read_csv(
    "../data/internal/SchoolYear.csv",
    usecols=["SchoolYearId", "SchoolYearNumberFall", "SchoolYearNumberSpring"],
)

school_year.head()

,SchoolYearId,SchoolYearNumberFall,SchoolYearNumberSpring
0,32,2021,2022
1,33,2022,2023
2,34,2023,2024
3,35,2024,2025
4,36,2025,2026


# School Dimension


In [4]:
school_dim = pd.read_csv(
    "../data/internal/DimSchool.csv",
    usecols=[
        "SchoolDetailFCSId",
        "SchoolDetailLevelDesc",
        "SchoolDetailReportName",
        "SchoolStatusDesc",
    ],
)

school_dim = school_dim[school_dim["SchoolStatusDesc"] == "Open"]
school_dim = school_dim.drop_duplicates(subset=["SchoolDetailFCSId"])
school_dim["SchoolDetailFCSId"] = school_dim["SchoolDetailFCSId"].astype(str)
school_dim.head()

,SchoolStatusDesc,SchoolDetailReportName,SchoolDetailFCSId,SchoolDetailLevelDesc
12,Open,High Point Elementary,228,Elementary School
13,Open,Mimosa Elementary,288,Elementary School
14,Open,Evoline C. West Elementary School,648,Elementary School
15,Open,College Park Elementary,48,Elementary School
16,Open,Brookview Elementary,54,Elementary School


# Enrolment Reason


In [5]:
enrolment_reason = pd.read_csv("../data/internal/EnrollmentReason.csv")
enrolment_reason.head()

,EnrollmentReasonId,EnrollmentReasonCode,EnrollmentReasonDesc,ActiveDimEnrollmentReasonRecordFlag
0,1,6,Displaced due to natural disaster,Y
1,2,A,Admitted from home school,Y
2,3,B,"Re-entered after withdrawal, this school this ...",Y
3,4,C,Continuing in same school,Y
4,5,D,Entered from a Department of Defense School,Y


# Withdrawal Reason


In [6]:
withdrawal_reason = pd.read_csv("../data/internal/WithdrawalReason.csv")
withdrawal_reason.head()

,WithdrawalReasonId,WithDrawalReasonCode,WithDrawalReasonDesc,ActiveDimWithdrawalReasonRecordFlag
0,1,YR,Year End,Y
1,2,1,SB10 Public Schools Transfer,Y
2,3,2,School Choice Transfer,Y
3,4,3,USCO,Y
4,5,4,Transferred Under the Jurisdiction of DJJ,Y


# Current Students


In [ ]:
enrolment = pd.read_csv(
    "../data/internal/Enrollment.csv",
    usecols=[
        "mask_studentpersonkey",
        "SchoolYearId",
        "SchoolDetailFCSId",
        "GradeLevel",
        "EnrollmentReasonId",
        "SchoolDetailFCSIdNextYear",
        "RetainedFlag",
        "CurrentEnrollment",
        "WithdrawalReasonId",
    ],
)

enrolment["mask_studentpersonkey"] = enrolment["mask_studentpersonkey"].astype(str)
enrolment["SchoolDetailFCSId"] = enrolment["SchoolDetailFCSId"].astype(str)
enrolment["SchoolYearId"] = enrolment["SchoolYearId"].astype(str)
enrolment["GradeLevel"] = enrolment["GradeLevel"].astype(str)
enrolment.head()

/var/folders/k2/dkmd0gld5bzg6w7mylrhgppr0000gn/T/ipykernel_38259/760980479.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  enrolment = pd.read_csv(


,SchoolYearId,mask_studentpersonkey,SchoolDetailFCSId,GradeLevel,EnrollmentReasonId,WithdrawalReasonId,CurrentEnrollment,RetainedFlag,SchoolDetailFCSIdNextYear
0,34,107055,693,6,14,1,N,N,693
1,35,107055,693,7,4,25,N,N,Unk
2,32,117995,852,12,4,13,N,N,Unk
3,32,114271,741,12,4,13,N,Y,Unk
4,32,117427,751,12,4,13,N,N,Unk


In [ ]:
enrolment = (
    pd.merge(enrolment, enrolment_reason, on=["EnrollmentReasonId"], how="left")
    .merge(withdrawal_reason, on=["WithdrawalReasonId"], how="left")
    .drop(
        columns=[
            "EnrollmentReasonId",
            "WithdrawalReasonId",
            "EnrollmentReasonCode",
            "WithDrawalReasonCode",
            "ActiveDimEnrollmentReasonRecordFlag",
            "ActiveDimWithdrawalReasonRecordFlag",
        ]
    )
)

enrolment.head()

,SchoolYearId,mask_studentpersonkey,SchoolDetailFCSId,GradeLevel,CurrentEnrollment,RetainedFlag,SchoolDetailFCSIdNextYear,EnrollmentReasonDesc,WithDrawalReasonDesc
0,34,107055,693,6,N,N,693,Transferred from another GA district,Year End
1,35,107055,693,7,N,N,Unk,Continuing in same school,Transfer to Another Public GA School
2,32,117995,852,12,N,N,Unk,Continuing in same school,High School Graduation
3,32,114271,741,12,N,Y,Unk,Continuing in same school,High School Graduation
4,32,117427,751,12,N,N,Unk,Continuing in same school,High School Graduation


# Graduation Summary


In [9]:
# Only for final year students
graduation_summary = pd.read_csv(
    "../data/internal/GraduationAreaSummary.csv",
    usecols=[
        "mask_studentpersonkey",
        "CurrentSchoolDetailFCSId",
        "SchoolYearNumberFall",
        "SubjectArea",
        "SubjectAreaCreditRequired",
        "AreaCredits",
        "AreaCreditStillNeeded",
    ],
)

graduation_summary.head()

,mask_studentpersonkey,CurrentSchoolDetailFCSId,SchoolYearNumberFall,SubjectArea,SubjectAreaCreditRequired,AreaCredits,AreaCreditStillNeeded
0,350335,741,2023,Math,4,1.5,2.5
1,1946451,870,2022,Health/ PersonalFitness,1,0.5,0.5
2,1987664,880,2024,World Language/ FineArts/ CareerTech,3,1.0,2.0
3,1521002,804,2024,World Language/ FineArts/ CareerTech,3,1.5,1.5
4,317341,751,2021,Social Studies,3,2.5,0.5


In [ ]:
graduation_summary = pd.merge(
    graduation_summary,
    school_year,
    on=["SchoolYearNumberFall"],
    how="left",
)

graduation_summary = graduation_summary.drop(
    columns=["SchoolYearNumberFall", "SchoolYearNumberSpring"]
).rename(columns={"CurrentSchoolDetailFCSId": "SchoolDetailFCSId"})

graduation_summary["mask_studentpersonkey"] = graduation_summary[
    "mask_studentpersonkey"
].astype(str)

graduation_summary["SchoolDetailFCSId"] = graduation_summary[
    "SchoolDetailFCSId"
].astype(str)

graduation_summary["SchoolYearId"] = graduation_summary["SchoolYearId"].astype(str)

grad_agg = graduation_summary.groupby(
    ["mask_studentpersonkey", "SchoolDetailFCSId", "SchoolYearId"]
).agg(list)

grad_agg.head()

,,,SubjectArea,SubjectAreaCreditRequired,AreaCredits,AreaCreditStillNeeded
mask_studentpersonkey,SchoolDetailFCSId,SchoolYearId,,,,
1389182,741,33,"[Electives, ELA, World Language/ FineArts/ Car...","[4, 4, 3, 4, 1, 3, 4]","[2.0, 3.0, 4.5, 2.5, 0.5, 2.0, 2.5]","[2.0, 1.0, 0.0, 1.5, 0.5, 1.0, 1.5]"
1390795,816,33,"[Math, ELA, Electives, World Language/ FineArt...","[4, 4, 4, 3, 4, 3]","[3.5, 2.5, 7.5, 7.5, 4.5, 1.5]","[0.5, 1.5, 0.0, 0.0, 0.0, 1.5]"
1390797,816,33,"[Math, Social Studies, ELA, Science, Electives...","[4, 3, 4, 4, 4, 3, 1]","[2.5, 1.5, 2.5, 3.0, 7.0, 5.5, 0.5]","[1.5, 1.5, 1.5, 1.0, 0.0, 0.0, 0.5]"
1392398,804,33,"[Electives, Health/ PersonalFitness, World Lan...","[4, 1, 3, 4, 4, 4, 3]","[3.0, 0.0, 4.5, 2.5, 2.5, 2.5, 1.5]","[1.0, 1.0, 0.0, 1.5, 1.5, 1.5, 1.5]"
1393973,743,34,"[Health/ PersonalFitness, Math, Social Studies...","[1, 4, 3, 4, 3, 4, 4]","[0.5, 1.5, 1.0, 1.5, 3.0, 0.0, 1.5]","[0.5, 2.5, 2.0, 2.5, 0.0, 4.0, 2.5]"


In [ ]:
grad_agg_nest = nest(grad_agg, "grad", grad_agg.columns)
grad_agg_nest.head()

,,,grad
mask_studentpersonkey,SchoolDetailFCSId,SchoolYearId,
1389182,741,33,"NestedData(SubjectArea=['Electives', 'ELA', 'W..."
1390795,816,33,"NestedData(SubjectArea=['Math', 'ELA', 'Electi..."
1390797,816,33,"NestedData(SubjectArea=['Math', 'Social Studie..."
1392398,804,33,"NestedData(SubjectArea=['Electives', 'Health/ ..."
1393973,743,34,NestedData(SubjectArea=['Health/ PersonalFitne...


# Georgia Milestone Scores


In [12]:
milestone_scores = pd.read_csv(
    "../data/internal/Georgia Milestones Scores.csv",
    usecols=[
        "StudentPersonKey_mask",
        "SchoolYearId",
        "SchoolDetailFCSId",
        "TestingDateId",
        "SubjectDesc",
        "TestGrade",
        "AchievementLevel",
        "LexileScore",
        "GradeConversionScore",
        "ScaleScore",
    ],
)

milestone_scores.head()

,SubjectDesc,SchoolYearId,TestingDateId,StudentPersonKey_mask,SchoolDetailFCSId,ScaleScore,AchievementLevel,GradeConversionScore,LexileScore,TestGrade
0,Algebra I,32,11319,356119,804,421,1,54.0,0.0,11
1,American Literature & Composition,32,11319,217864,852,497,2,73.0,1165.0,11
2,American Literature & Composition,32,11319,237883,852,597,4,92.0,1800.0,12
3,Algebra I,32,11319,236191,852,493,2,72.0,0.0,12
4,Biology,32,11319,196546,852,464,1,65.0,0.0,11


In [37]:
milestone_scores = milestone_scores.rename(
    columns={"StudentPersonKey_mask": "mask_studentpersonkey"}
)

milestone_scores["mask_studentpersonkey"] = milestone_scores[
    "mask_studentpersonkey"
].astype(str)

milestone_scores["SchoolDetailFCSId"] = milestone_scores["SchoolDetailFCSId"].astype(
    str
)

milestone_scores["SchoolYearId"] = milestone_scores["SchoolYearId"].astype(str)

milestone_agg = (
    milestone_scores.sort_values(
        by=[
            "mask_studentpersonkey",
            "SchoolDetailFCSId",
            "SchoolYearId",
            "TestingDateId",
        ]
    )
    .groupby(["mask_studentpersonkey", "SchoolDetailFCSId", "SchoolYearId"])
    .agg(list)
)

milestone_agg.head()

SubjectDesc  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                                         
107055                693               34            [English Language Arts, Mathematics]   
1387704               693               32            [English Language Arts, Mathematics]   
                                        33            [English Language Arts, Mathematics]   
1389182               741               33                                       [Biology]   
1389243               695               33            [English Language Arts, Mathematics]   

                                                       TestingDateId  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                   
107055                693               34            [12299, 12299]   
1387704               693               32            [11571, 11571]   
                                        33            [11935, 11935]   
1389182               741               33                   [11942]   
1389243               695               33            [11935, 11935]   

                                                      ScaleScore  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId               
107055                693               34            [490, 528]   
1387704               693               32            [566, 481]   
                                        33            [508, 499]   
1389182               741               33                 [623]   
1389243               695               33            [513, 524]   

                                                     AchievementLevel  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                    
107055                693               34                     [2, 3]   
1387704               693               32                     [3, 2]   
                                        33                     [2, 2]   
1389182               741               33                        [4]   
1389243               695               33                     [2, 2]   

                                                     GradeConversionScore  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                        
107055                693               34                     [nan, nan]   
1387704               693               32                     [nan, nan]   
                                        33                     [nan, nan]   
1389182               741               33                         [93.0]   
1389243               695               33                     [nan, nan]   

                                                        LexileScore TestGrade  
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                           
107055                693               34             [760.0, nan]    [6, 6]  
1387704               693               32            [1155.0, nan]    [6, 6]  
                                        33            [1095.0, nan]    [7, 7]  
1389182               741               33                    [0.0]       [9]  
1389243               695               33            [1095.0, nan]    [7, 7]

In [38]:
milestone_agg_nest = nest(milestone_agg, "milestone", milestone_agg.columns)
milestone_agg_nest.head()

milestone
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                                                   
107055                693               34            NestedData(SubjectDesc=['English Language Arts...
1387704               693               32            NestedData(SubjectDesc=['English Language Arts...
                                        33            NestedData(SubjectDesc=['English Language Arts...
1389182               741               33            NestedData(SubjectDesc=['Biology'], TestingDat...
1389243               695               33            NestedData(SubjectDesc=['English Language Arts...

# SAT Scores


In [15]:
sat_scores = pd.read_csv(
    "../data/internal/Fact_SATSummaryTest.csv",
    usecols=[
        "mask_studentpersonkey",
        "SchoolDetailFCSId",
        "TestingDateId",
        "MathScore",
        "VerbalScore",
        "TotalScore",
        "MathPercentile",
        "VerbalPercentile",
    ],
)

sat_scores.head()

,TestingDateId,MathScore,VerbalScore,TotalScore,MathPercentile,VerbalPercentile,mask_studentpersonkey,SchoolDetailFCSId
0,10652,730,640,1370,97,88,220288,751
1,11016,730,660,1390,97,92,306968,754
2,11040,370,380,750,9,11,430528,914
3,11051,750,690,1440,98,96,219659,751
4,11051,640,640,1280,89,88,218026,751


In [ ]:
sat_scores["mask_studentpersonkey"] = sat_scores["mask_studentpersonkey"].astype(str)
sat_scores["SchoolDetailFCSId"] = sat_scores["SchoolDetailFCSId"].astype(str)

sat_agg = (
    sat_scores.sort_values(
        by=["mask_studentpersonkey", "SchoolDetailFCSId", "TestingDateId"]
    )
    .groupby(["mask_studentpersonkey", "SchoolDetailFCSId"])
    .agg(list)
)

sat_agg.head()

,,TestingDateId,MathScore,VerbalScore,TotalScore,MathPercentile,VerbalPercentile
mask_studentpersonkey,SchoolDetailFCSId,,,,,,
100075,740,[11391],[350],[410],[760],[5],[19]
104981,852,[11261],[520],[500],[1020],[57],[48]
120717,845,[12542],[520],[640],[1160],[57],[88]
128475,740,[11961],[530],[530],[1060],[61],[58]
134389,751,"[12542, 12633]","[740, 730]","[470, 550]","[1210, 1280]","[98, 97]","[38, 65]"


In [ ]:
sat_agg_nest = nest(sat_agg, "sat", sat_agg.columns)
sat_agg_nest.head()

,,sat
mask_studentpersonkey,SchoolDetailFCSId,
100075,740,"NestedData(TestingDateId=[11391], MathScore=[3..."
104981,852,"NestedData(TestingDateId=[11261], MathScore=[5..."
120717,845,"NestedData(TestingDateId=[12542], MathScore=[5..."
128475,740,"NestedData(TestingDateId=[11961], MathScore=[5..."
134389,751,"NestedData(TestingDateId=[12542, 12633], MathS..."


# Illuminate Data


In [18]:
illu_2022 = pd.read_csv(
    "../data/internal/IlluminateData2022.csv",
    encoding="latin-1",
    usecols=[
        "Mask_StudentPersonkey",
        "schoolyearnumberspring",
        "CurrentSchoolDetailFCSID",
        "AssessmentGradeLevel",
        "GradeLevelDuringUnitTest",
        "Standard_Subject",
        "responsedatevalue",
        "condition",
        "Response_points",
        "Response_points_possible",
        "Response_percent_correct",
        "Standard_points",
        "Standard_points_possible",
        "Standard_percent_correct",
        "ContinuousAchievementLevel",
    ],
)

illu_2022.head()

/var/folders/k2/dkmd0gld5bzg6w7mylrhgppr0000gn/T/ipykernel_38259/3631683817.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  illu_2022 = pd.read_csv(


,schoolyearnumberspring,Mask_StudentPersonkey,responsedatevalue,Response_points,Response_points_possible,Response_percent_correct,Standard_Subject,Standard_points,Standard_points_possible,Standard_percent_correct,condition,GradeLevelDuringUnitTest,ContinuousAchievementLevel,AssessmentGradeLevel,CurrentSchoolDetailFCSID
0,2022,763538,2021-09-19 00:00:00,5.0,15.0,33.33,Mathematics,1.0,7.0,14.29,Reteach,3.0,On Level,3,492
1,2022,862425,2022-05-11 00:00:00,6.0,8.0,75.00,English Language Arts,1.0,1.0,100.00,Extension,6.0,On Level,6,696
2,2022,582651,2022-01-28 00:00:00,16.0,17.0,94.12,Mathematics,2.0,2.0,100.00,Extension,3.0,On Level,3,615
3,2022,584570,2021-10-29 00:00:00,7.0,10.0,70.00,English Language Arts,1.0,1.0,100.00,Extension,4.0,On Level,4,120
4,2022,550130,2022-02-23 00:00:00,13.0,15.0,86.67,Mathematics,2.0,2.0,100.00,Extension,7.0,Accelerated,8,686


In [ ]:
illu_2023 = pd.read_csv(
    "../data/internal/IlluminateData2023.csv",
    encoding="latin-1",
    usecols=[
        "Mask_StudentPersonkey",
        "schoolyearnumberspring",
        "CurrentSchoolDetailFCSID",
        "AssessmentGradeLevel",
        "GradeLevelDuringUnitTest",
        "Standard_Subject",
        "responsedatevalue",
        "condition",
        "Response_points",
        "Response_points_possible",
        "Response_percent_correct",
        "Standard_points",
        "Standard_points_possible",
        "Standard_percent_correct",
        "ContinuousAchievementLevel",
    ],
)

illu_2023.head()

,schoolyearnumberspring,Mask_StudentPersonkey,responsedatevalue,Response_points,Response_points_possible,Response_percent_correct,Standard_Subject,Standard_points,Standard_points_possible,Standard_percent_correct,condition,GradeLevelDuringUnitTest,ContinuousAchievementLevel,AssessmentGradeLevel,CurrentSchoolDetailFCSID
0,2023,405692,2022-10-20 00:00:00,13.0,13.0,100.00,Mathematics,1.0,1.0,100.0,Extension,05,On Level,5,188
1,2023,772909,2023-02-24 00:00:00,12.0,15.0,80.00,Mathematics,3.0,3.0,100.0,Extension,08,On Level,8,706
2,2023,710128,2023-05-17 00:00:00,10.0,10.0,100.00,English Language Arts,1.0,1.0,100.0,Extension,05,On Level,5,672
3,2023,581938,2022-12-08 00:00:00,14.0,14.0,100.00,Mathematics,4.0,4.0,100.0,Extension,01,On Level,1,270
4,2023,459392,2022-11-18 00:00:00,9.0,17.0,52.94,Mathematics,1.0,2.0,50.0,Reteach,08,On Level,8,695


In [ ]:
illu_2024 = pd.read_csv(
    "../data/internal/IlluminateData2024.csv",
    encoding="latin-1",
    usecols=[
        "Mask_StudentPersonkey",
        "schoolyearnumberspring",
        "CurrentSchoolDetailFCSID",
        "AssessmentGradeLevel",
        "GradeLevelDuringUnitTest",
        "Standard_Subject",
        "responsedatevalue",
        "condition",
        "Response_points",
        "Response_points_possible",
        "Response_percent_correct",
        "Standard_points",
        "Standard_points_possible",
        "Standard_percent_correct",
        "ContinuousAchievementLevel",
    ],
)

illu_2024.head()

,schoolyearnumberspring,Mask_StudentPersonkey,responsedatevalue,Response_points,Response_points_possible,Response_percent_correct,Standard_Subject,Standard_points,Standard_points_possible,Standard_percent_correct,condition,GradeLevelDuringUnitTest,ContinuousAchievementLevel,AssessmentGradeLevel,CurrentSchoolDetailFCSID
0,2024,416890,2023-09-22 00:00:00,10.0,15.0,66.67,Georgias K-12 Mathematics Standards,0.0,1.0,0.0,Reteach,06,On Level,6,692
1,2024,762380,2023-11-03 00:00:00,7.0,17.0,41.18,English Language Arts,1.0,1.0,100.0,Extension,11,Not Applicable,11,914
2,2024,712301,2023-10-31 00:00:00,5.0,13.0,38.46,English Language Arts,0.0,1.0,0.0,Reteach,05,Not Applicable,5,980
3,2024,472942,2024-02-08 00:00:00,12.0,13.0,92.31,English Language Arts,1.0,1.0,100.0,Extension,03,On Level,3,607
4,2024,575441,2023-09-14 00:00:00,9.0,10.0,90.00,English Language Arts,1.0,1.0,100.0,Extension,04,Advanced,4,607


In [ ]:
illu_2025 = pd.read_csv(
    "../data/internal/IlluminateData2025.csv",
    encoding="latin-1",
    usecols=[
        "Mask_StudentPersonkey",
        "schoolyearnumberspring",
        "CurrentSchoolDetailFCSID",
        "AssessmentGradeLevel",
        "GradeLevelDuringUnitTest",
        "Standard_Subject",
        "responsedatevalue",
        "condition",
        "Response_points",
        "Response_points_possible",
        "Response_percent_correct",
        "Standard_points",
        "Standard_points_possible",
        "Standard_percent_correct",
        "ContinuousAchievementLevel",
    ],
)

illu_2025.head()

,schoolyearnumberspring,Mask_StudentPersonkey,responsedatevalue,Response_points,Response_points_possible,Response_percent_correct,Standard_Subject,Standard_points,Standard_points_possible,Standard_percent_correct,condition,GradeLevelDuringUnitTest,ContinuousAchievementLevel,AssessmentGradeLevel,CurrentSchoolDetailFCSID
0,2025,406954,2025-01-13 00:00:00,10.0,14.0,71.43,English Language Arts,2.0,2.0,100.0,Extension,06,On Level,6,682
1,2025,405935,2024-10-14 00:00:00,10.0,14.0,71.43,Georgias K-12 Mathematics Standards,2.0,2.0,100.0,Extension,06,On Level,6,697
2,2025,728618,2025-02-14 00:00:00,12.0,17.0,70.59,English Language Arts,1.0,1.0,100.0,Extension,09,On Level,10,845
3,2025,728056,2024-12-19 00:00:00,11.5,15.0,76.67,Georgias K-12 Mathematics Standards,3.0,3.0,100.0,Extension,06,Advanced,7,699
4,2025,728145,2025-02-11 00:00:00,12.5,13.0,96.15,English Language Arts,1.0,1.0,100.0,Extension,03,On Level,3,654


In [ ]:
illu = pd.concat([illu_2022, illu_2023, illu_2024, illu_2025], axis=0)
illu.head()

,schoolyearnumberspring,Mask_StudentPersonkey,responsedatevalue,Response_points,Response_points_possible,Response_percent_correct,Standard_Subject,Standard_points,Standard_points_possible,Standard_percent_correct,condition,GradeLevelDuringUnitTest,ContinuousAchievementLevel,AssessmentGradeLevel,CurrentSchoolDetailFCSID
0,2022,763538,2021-09-19 00:00:00,5.0,15.0,33.33,Mathematics,1.0,7.0,14.29,Reteach,3.0,On Level,3,492
1,2022,862425,2022-05-11 00:00:00,6.0,8.0,75.00,English Language Arts,1.0,1.0,100.00,Extension,6.0,On Level,6,696
2,2022,582651,2022-01-28 00:00:00,16.0,17.0,94.12,Mathematics,2.0,2.0,100.00,Extension,3.0,On Level,3,615
3,2022,584570,2021-10-29 00:00:00,7.0,10.0,70.00,English Language Arts,1.0,1.0,100.00,Extension,4.0,On Level,4,120
4,2022,550130,2022-02-23 00:00:00,13.0,15.0,86.67,Mathematics,2.0,2.0,100.00,Extension,7.0,Accelerated,8,686


In [23]:
def standardize_grade_level_values(grade_value):
    """Standardize grade level values to consistent format"""
    if pd.isna(grade_value):
        return None

    # Convert to string and clean
    grade_str = str(grade_value).strip()

    # Handle special cases first
    if grade_str.upper() in ["KINDERGARTEN", "KK", "K"]:
        return "K"
    elif grade_str.upper() in ["PK", "PRE-K", "PREK"]:
        return "PK"
    elif grade_str.upper() in ["HS", "HIGH SCHOOL"]:
        return "HS"

    # Handle numeric grades - ensure they're single digits for K-12
    try:
        # Remove any leading zeros and convert to integer
        grade_num = int(float(grade_str))
        if 1 <= grade_num <= 12:
            return str(grade_num)
    except:
        pass

    # Return original if can't standardize
    return grade_str

In [24]:
# fmt:off
illu["AssessmentGradeLevel"] = illu["AssessmentGradeLevel"].apply(standardize_grade_level_values)
illu["GradeLevelDuringUnitTest"] = illu["GradeLevelDuringUnitTest"].apply(standardize_grade_level_values)
# fmt:on

In [25]:
illu["responsedatevalue"] = pd.to_datetime(illu["responsedatevalue"]).dt.date

illu = illu.rename(
    columns={
        "schoolyearnumberspring": "SchoolYearNumberSpring",
        "Mask_StudentPersonkey": "mask_studentpersonkey",
        "CurrentSchoolDetailFCSID": "SchoolDetailFCSId",
    }
)

illu = pd.merge(illu, school_year, on=["SchoolYearNumberSpring"], how="left")
illu = illu.drop(columns=["SchoolYearNumberFall", "SchoolYearNumberSpring"])
illu.head()

,mask_studentpersonkey,responsedatevalue,Response_points,Response_points_possible,Response_percent_correct,Standard_Subject,Standard_points,Standard_points_possible,Standard_percent_correct,condition,GradeLevelDuringUnitTest,ContinuousAchievementLevel,AssessmentGradeLevel,SchoolDetailFCSId,SchoolYearId
0,763538,2021-09-19,5.0,15.0,33.33,Mathematics,1.0,7.0,14.29,Reteach,3,On Level,3,492,32
1,862425,2022-05-11,6.0,8.0,75.00,English Language Arts,1.0,1.0,100.00,Extension,6,On Level,6,696,32
2,582651,2022-01-28,16.0,17.0,94.12,Mathematics,2.0,2.0,100.00,Extension,3,On Level,3,615,32
3,584570,2021-10-29,7.0,10.0,70.00,English Language Arts,1.0,1.0,100.00,Extension,4,On Level,4,120,32
4,550130,2022-02-23,13.0,15.0,86.67,Mathematics,2.0,2.0,100.00,Extension,7,Accelerated,8,686,32


In [26]:
illu["mask_studentpersonkey"] = illu["mask_studentpersonkey"].astype(str)
illu["SchoolDetailFCSId"] = illu["SchoolDetailFCSId"].astype(str)
illu["SchoolYearId"] = illu["SchoolYearId"].astype(str)

illu_agg = (
    illu.sort_values(
        by=[
            "mask_studentpersonkey",
            "SchoolDetailFCSId",
            "SchoolYearId",
            "responsedatevalue",
        ]
    )
    .groupby(["mask_studentpersonkey", "SchoolDetailFCSId", "SchoolYearId"])
    .agg(list)
)

illu_agg.head()

responsedatevalue  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                                                      
107055                693               34            [2024-02-08, 2024-02-08, 2024-02-08, 2024-02-0...   
1387704               693               32            [2022-03-16, 2022-03-16, 2022-03-16, 2022-03-1...   
                                        33            [2023-03-08, 2023-03-08, 2023-03-08, 2023-03-0...   
1389218               601               33            [2022-09-19, 2022-09-19, 2022-09-19, 2022-09-1...   
                                        34            [2023-09-22, 2023-09-22, 2023-09-22, 2023-09-2...   

                                                                                        Response_points  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                                                      
107055                693               34            [7.5, 7.5, 7.5, 7.5, 3.67, 3.67, 3.67, 8.0, 8....   
1387704               693               32            [4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 7.0, ...   
                                        33            [9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 7.0, 7.0, ...   
1389218               601               33            [21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21....   
                                        34            [13.0, 13.0, 13.0, 13.0, 13.0, 12.0, 12.0, 12....   

                                                                               Response_points_possible  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                                                      
107055                693               34            [13.0, 13.0, 13.0, 13.0, 10.0, 10.0, 10.0, 16....   
1387704               693               32            [13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13....   
                                        33            [14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 14.0, 11....   
1389218               601               33            [35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35.0, 35....   
                                        34            [18.0, 18.0, 18.0, 18.0, 18.0, 15.0, 15.0, 15....   

                                                                               Response_percent_correct  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                                                      
107055                693               34            [57.69, 57.69, 57.69, 57.69, 36.7, 36.7, 36.7,...   
1387704               693               32            [30.77, 30.77, 30.77, 30.77, 30.77, 30.77, 30....   
                                        33            [64.29, 64.29, 64.29, 64.29, 64.29, 64.29, 64....   
1389218               601               33            [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60....   
                                        34            [72.22, 72.22, 72.22, 72.22, 72.22, 80.0, 80.0...   

                                                                                       Standard_Subject  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                                                      
107055                693               34            [Georgias K-12 Mathematics Standards, Georgia...   
1387704               693               32            [Mathematics, Mathematics, Mathematics, Mathem...   
                                        33            [English Language Arts, English Language Arts,...   
1389218               601               33            [Mathematics, Mathematics, Mathematics, Mathem...   
                                        34            [Georgias K-12 Mathematics Standards, Georgia...   

                                                                                        Standard_points  \
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                                                      
107055                693               34            [5.0, 2.0, 0.5, 1.0, 1.67, 2.0, 0.0, 0.0, 2.0,...   
1387704        

In [27]:
illu_agg_nest = nest(illu_agg, "illuminate", illu_agg.columns)
illu_agg_nest.head()

illuminate
mask_studentpersonkey SchoolDetailFCSId SchoolYearId                                                   
107055                693               34            NestedData(responsedatevalue=[datetime.date(20...
1387704               693               32            NestedData(responsedatevalue=[datetime.date(20...
                                        33            NestedData(responsedatevalue=[datetime.date(20...
1389218               601               33            NestedData(responsedatevalue=[datetime.date(20...
                                        34            NestedData(responsedatevalue=[datetime.date(20...

# MAP Subject RIT Score


In [ ]:
map_rit_scores = pd.read_csv(
    "../data/internal/Map Subject Test Data.csv",
    usecols=[
        "mask_studentpersonkey",
        "SchoolDetailFCSId",
        "TestingDateId",
        "TermName",
        "TestDurationMinutes",
        "TestRITScore",
        "TestPercentile",
        "PercentCorrect",
        "AchievementQuintile",
    ],
)

map_rit_scores.head()

,TestingDateId,mask_studentpersonkey,SchoolDetailFCSId,TermName,TestDurationMinutes,TestRITScore,TestPercentile,AchievementQuintile,PercentCorrect
0,11756,372350,687,Fall 2022-2023,93,185,3,Low,45
1,11756,418318,687,Fall 2022-2023,59,179,1,Low,26
2,11756,779939,687,Fall 2022-2023,89,214,47,Avg,48
3,11756,371484,687,Fall 2022-2023,144,208,33,LoAvg,53
4,11763,332500,687,Fall 2022-2023,18,184,3,Low,30


In [ ]:
map_rit_scores["mask_studentpersonkey"] = map_rit_scores[
    "mask_studentpersonkey"
].astype(str)

map_rit_scores["SchoolDetailFCSId"] = map_rit_scores["SchoolDetailFCSId"].astype(str)

rit_agg = (
    map_rit_scores.sort_values(
        ["mask_studentpersonkey", "SchoolDetailFCSId", "TestingDateId"]
    )
    .groupby(["mask_studentpersonkey", "SchoolDetailFCSId"])
    .agg(list)
)

rit_agg.head()

,,TestingDateId,TermName,TestDurationMinutes,TestRITScore,TestPercentile,AchievementQuintile,PercentCorrect
mask_studentpersonkey,SchoolDetailFCSId,,,,,,,
133481,870,"[11713, 11713]","[Fall 2022-2023, Fall 2022-2023]","[25, 33]","[172, 172]","[1, 1]","[Low, Low]","[26, 33]"
1389182,741,"[11700, 11700, 11848, 12068, 12069, 12449]","[Fall 2022-2023, Fall 2022-2023, Winter 2022-2...","[65, 77, 34, 56, 75, 70]","[241, 253, 232, 239, 247, 248]","[88, 91, 57, 84, 81, 79]","[High, High, Avg, High, High, HiAvg]","[50, 53, 49, 53, 49, 51]"
1389243,695,"[11749, 11763]","[Fall 2022-2023, Fall 2022-2023]","[47, 22]","[225, 214]","[58, 47]","[Avg, Avg]","[51, 39]"
1389244,754,"[11704, 11707]","[Fall 2022-2023, Fall 2022-2023]","[44, 36]","[231, 232]","[74, 61]","[HiAvg, HiAvg]","[55, 53]"
1389249,621,"[12103, 12110]","[Fall 2023-2024, Fall 2023-2024]","[61, 36]","[188, 194]","[80, 90]","[HiAvg, High]","[56, 54]"


In [30]:
rit_agg_nest = nest(rit_agg, "rit", rit_agg.columns)
rit_agg_nest.head()

,,rit
mask_studentpersonkey,SchoolDetailFCSId,
133481,870,"NestedData(TestingDateId=[11713, 11713], TermN..."
1389182,741,"NestedData(TestingDateId=[11700, 11700, 11848,..."
1389243,695,"NestedData(TestingDateId=[11749, 11763], TermN..."
1389244,754,"NestedData(TestingDateId=[11704, 11707], TermN..."
1389249,621,"NestedData(TestingDateId=[12103, 12110], TermN..."


# Merge


In [ ]:
merged = (
    pd.merge(
        enrolment,
        grad_agg_nest,
        on=["mask_studentpersonkey", "SchoolDetailFCSId", "SchoolYearId"],
        how="outer",
    )
    .merge(
        milestone_agg_nest,
        on=["mask_studentpersonkey", "SchoolDetailFCSId", "SchoolYearId"],
        how="outer",
    )
    .merge(
        illu_agg_nest,
        on=["mask_studentpersonkey", "SchoolDetailFCSId", "SchoolYearId"],
        how="outer",
    )
    .merge(
        sat_agg_nest,
        on=["mask_studentpersonkey", "SchoolDetailFCSId"],
        how="outer",
    )
    .merge(
        rit_agg_nest,
        on=["mask_studentpersonkey", "SchoolDetailFCSId"],
        how="outer",
    )
)

merged.head()

,SchoolYearId,mask_studentpersonkey,SchoolDetailFCSId,GradeLevel,CurrentEnrollment,RetainedFlag,SchoolDetailFCSIdNextYear,EnrollmentReasonDesc,WithDrawalReasonDesc,grad,milestone,illuminate,sat,rit
0,NaN,100075,740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NestedData(TestingDateId=[11391], MathScore=[3...",NaN
1,NaN,104981,852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NestedData(TestingDateId=[11261], MathScore=[5...",NaN
2,34,107055,693,6,N,N,693,Transferred from another GA district,Year End,NaN,NestedData(SubjectDesc=['English Language Arts...,NestedData(responsedatevalue=[datetime.date(20...,NaN,NaN
3,35,107055,693,7,N,N,Unk,Continuing in same school,Transfer to Another Public GA School,NaN,NaN,NaN,NaN,NaN
4,32,114271,741,12,N,Y,Unk,Continuing in same school,High School Graduation,NaN,NaN,NaN,NaN,NaN


In [43]:
def analyze_merged_dataset(df):
    """Comprehensive analysis of the merged student dataset"""

    print("\n" + "=" * 70)
    print("COMPREHENSIVE STUDENT DATASET ANALYSIS")
    print("=" * 70)

    # Basic metrics
    total_records = len(df)
    unique_students = df["mask_studentpersonkey"].nunique()

    print(f"\nDATASET OVERVIEW:")
    print(f"   Total Records: {total_records:,}")
    print(f"   Unique Students: {unique_students:,}")
    print(f"   Average Records per Student: {total_records/unique_students:.2f}")

    # Assessment coverage analysis
    assessments = {
        "Graduation Data": "grad",
        "GA Milestones": "milestone",
        "Illuminate": "illuminate",
        "SAT Scores": "sat",
        "RIT Scores": "rit",
    }

    print(f"\nASSESSMENT DATA COVERAGE:")
    print(
        f"{'Assessment':<17} {'Records':<10} {'%':<8} {'Students':<10} {'Student %':<10}"
    )
    print("-" * 65)

    for name, col in assessments.items():
        records_with_data = df[~df[col].isnull()]
        record_count = len(records_with_data)
        record_pct = (record_count / total_records) * 100
        student_count = records_with_data["mask_studentpersonkey"].nunique()
        student_pct = (student_count / unique_students) * 100

        print(
            f"{name:<17} {record_count:<10,} {record_pct:<7.1f}% {student_count:<10,} {student_pct:<9.1f}%"
        )

    # Students with multiple assessment types
    print(f"\nMULTI-ASSESSMENT COVERAGE:")
    assessment_cols = list(assessments.values())
    student_coverage = df.groupby("mask_studentpersonkey")[assessment_cols].apply(
        lambda x: x.notna().sum(axis=1).max()
    )

    for i in range(1, len(assessment_cols) + 1):
        count = (student_coverage >= i).sum()
        pct = (count / unique_students) * 100
        print(f"   Students with {i}+ assessments: {count:,} ({pct:.1f}%)")

    print("=" * 70)


# Call the analysis
analyze_merged_dataset(merged)


COMPREHENSIVE STUDENT DATASET ANALYSIS

DATASET OVERVIEW:
   Total Records: 603,464
   Unique Students: 159,092
   Average Records per Student: 3.79

ASSESSMENT DATA COVERAGE:
Assessment        Records    %        Students   Student % 
-----------------------------------------------------------------
Graduation Data   29,859     4.9    % 29,316     18.4     %
GA Milestones     175,291    29.0   % 86,976     54.7     %
Illuminate        280,369    46.5   % 120,820    75.9     %
SAT Scores        55,427     9.2    % 21,137     13.3     %
RIT Scores        268,379    44.5   % 82,381     51.8     %

MULTI-ASSESSMENT COVERAGE:
   Students with 1+ assessments: 134,768 (84.7%)
   Students with 2+ assessments: 95,458 (60.0%)
   Students with 3+ assessments: 68,359 (43.0%)
   Students with 4+ assessments: 20,994 (13.2%)
   Students with 5+ assessments: 3,548 (2.2%)
